# Gap up
**Truth Premise: When a day ends and a new day begins in life of options premium, its value depreciates because of time decay because it gets closer to expiry and the probability of its contract going as expected by the buyer gets lesser.**

In [1]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
from functools import cache
from constants import *


# TEST_START = dt.datetime.strptime("2019-09-01", "%Y-%m-%d")
TEST_START = dt.datetime.strptime("2018-08-24", "%Y-%m-%d")
TEST_START = dt.datetime.strptime("2021-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2020-12-31", "%Y-%m-%d")
# TEST_END = dt.datetime.strptime("2019-09-30", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2024-02-29", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE
MIN_MA_GAP = 10
sma_length = 9
# DAILY_INVESTMENT = 1 * (10 ** 5) # 1 lakh
LOT_SIZE = 50
LOT_QTY = 10

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)
# pd.set_option('precision', 2)
pd.set_option("display.precision", 2)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

def build_date_range(date_start, date_end, symbol):
    date_range = []
    cur_date = date_start
    while cur_date < date_end:
        if cur_date.weekday() not in [5, 6]:
            has_data, _ = ut.has_data(symbol, cur_date, interval=INTERVAL, exchange=EXCHANGE)
            if has_data:
                date_range.append(cur_date)
        cur_date += dt.timedelta(days=1)
    return date_range

all_dates = pd.DataFrame({"trade_date": build_date_range(TEST_START, TEST_END, SYMBOL)})
all_dates_shuffled = all_dates.sample(frac=1, random_state=42)

train_size = int(0.5 * len(all_dates_shuffled))
train_dates = all_dates_shuffled.iloc[:train_size]
test_dates = all_dates_shuffled.iloc[train_size:]
train_dates = all_dates
train_dates = train_dates.sort_values(by="trade_date")
train_dates.set_index("trade_date", inplace=True)
test_dates = test_dates.sort_values(by="trade_date")
test_dates.set_index("trade_date", inplace=True)



@cache
def get_intraday_data(date):
    return ut.get_data(symbol=SYMBOL, date=date, interval=INTERVAL, exchange=EXCHANGE)

def get_daily_data(date):
    return ut.get_data(symbol=SYMBOL, date=date, interval=INTERVAL_DAY, exchange=EXCHANGE)

@cache
def get_symbol_first_candle(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].open, data.iloc[0].high, data.iloc[0].low, data.iloc[0].close

@cache
def get_first_candle_close(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].close

@cache
def get_last_trading_day(date):
    return ut.get_last_trading_day(SYMBOL, date, interval=INTERVAL, exchange=ut.EXCHANGE_NSE)

expiry = 1
train_dates["previous_trading_day"] = None
train_dates["previous_trading_day"] = train_dates.apply(lambda row: get_last_trading_day(row.name), axis=1)
train_dates["expiry"] = train_dates.apply(lambda row: ut.find_nclosest_expiry(SYMBOL, row.name, expiry), axis=1)

def get_nifty_price(d, t):
    data = ut.get_data(symbol=SYMBOL, date=d, interval=INTERVAL, exchange=EXCHANGE)
    try:
        return data.loc[data.index.time == t].iloc[0].open
    except IndexError:
        return pd.NA
    except AttributeError as e:
        return pd.NA

buy_time = dt.time(hour=15, minute=28)
sell_time = dt.time(hour=9, minute=16)

train_dates = train_dates.copy()

# train_dates["trade_day_before_expiry"] = train_dates.expiry - pd.Timedelta(days=day_before_expiry)
train_dates.loc[:, "nifty_at_0320"] = train_dates.apply(lambda r: get_nifty_price(r.previous_trading_day.date(), dt.time(hour=15, minute=20)), axis=1)
train_dates.loc[:, "td_nifty_at_0918"] = train_dates.apply(lambda r: get_nifty_price(r.name.date(), dt.time(hour=9, minute=15)), axis=1)
train_dates.loc[:, "td_nifty_at_0328"] = train_dates.apply(lambda r: get_nifty_price(r.previous_trading_day.date(), dt.time(hour=15, minute=29)), axis=1)
train_dates.loc[:, "td_nifty_diff"] = train_dates.td_nifty_at_0328 - train_dates.td_nifty_at_0918
train_dates.loc[:, "nifty_at_buy"] = train_dates.apply(lambda r: get_nifty_price(r.previous_trading_day.date(), buy_time), axis=1)
train_dates.loc[:, "nifty_at_sell"] = train_dates.apply(lambda r: get_nifty_price(r.name.date(), sell_time), axis=1)
train_dates.loc[:, "nifty_diff"] = train_dates.nifty_at_sell - train_dates.nifty_at_buy

nifty_candles = train_dates.previous_trading_day.dt.date.apply(get_daily_data).to_list()
# nifty_candles = pd.DataFrame(nifty_candles)
nifty_candles = pd.concat(nifty_candles)
sma_key = f"sma{sma_length}"
nifty_candles[sma_key] = nifty_candles["close"].rolling(window=sma_length).mean()

train_dates = train_dates.loc[train_dates.nifty_at_0320.notna()]
train_dates["atm_strike"] = train_dates.apply(lambda trade: round(trade.nifty_at_0320 / 50) * 50, axis=1)

In [2]:
# pd.set_option("display.max_colwidth", None)
# pd.set_option("display.max_rows", 200)

# nifty_candles
# train_dates.previous_trading_day.max()

# nifty_candles
# nifty_candles.loc[nifty_candles.index.date == train_dates.iloc[11].previous_trading_day.date()]
train_dates

,previous_trading_day,expiry,nifty_at_0320,td_nifty_at_0918,td_nifty_at_0328,td_nifty_diff,nifty_at_buy,nifty_at_sell,nifty_diff,atm_strike
trade_date,,,,,,,,,,
2018-08-24,2018-08-23,2018-08-30,11589.1,11566.6,11579.8,13.2,11581.1,11568.65,-12.45,11600
2018-08-27,2018-08-24,2018-08-30,11558.15,11605.85,11554.1,-51.75,11554.7,11627.2,72.5,11550
2018-08-28,2018-08-27,2018-08-30,11691.5,11731.95,11696.1,-35.85,11697.25,11743.5,46.25,11700
2018-08-29,2018-08-28,2018-08-30,11744.3,11744.95,11740.25,-4.7,11739.65,11730.05,-9.6,11750
2018-08-30,2018-08-29,2018-08-30,11688.0,11694.75,11687.6,-7.15,11688.65,11696.2,7.55,11700
2018-08-31,2018-08-30,2018-09-27,11680.85,11675.85,11684.05,8.2,11682.9,11658.55,-24.35,11700
2018-09-03,2018-08-31,2018-09-27,11682.0,11751.8,11675.35,-76.45,11677.9,11700.3,22.4,11700
2018-09-04,2018-09-03,2018-09-27,11575.3,11598.75,11571.5,-27.25,11573.7,11591.35,17.65,11600
2018-09-05,2018-09-04,2018-09-27,11521.2,11514.85,11522.85,8.0,11520.25,11526.45,6.2,11500


In [3]:
def get_premium_df(trade, strike_price, td, option_type, tm):
    pr = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=trade.expiry, cur_dt=td, strike_price=strike_price, option_type=option_type)
    if type(pr) == type(pd.NA) or pr.shape[0] == 0:
        return pd.NA
    x = pr.loc[(pr.index.date == td) & (pr.index.time == tm)]
    # # print(x.iloc[0])
    if x.shape[0] == 0:
        # print(tm)
        # print(td)
        # print(pr.loc[(pr.index.date == td)])
        # print(trade)
        return pd.NA
    return x.iloc[0].close

def get_next_1000(strike, i):
    divider = 50
    reminder = strike % divider
    if reminder != 0:
        return divider * (strike // divider) - i * divider
    return divider * (strike // divider) - (i+1) * divider


train_dates["day_sma"] = nifty_candles.loc[train_dates.previous_trading_day.dt.date.values][sma_key].values
train_dates["sma_favors"] = train_dates.day_sma < (train_dates.nifty_at_buy - MIN_MA_GAP)
train_dates = train_dates.loc[train_dates.sma_favors].copy()
train_dates.dropna(inplace=True)

/home/nitin/anaconda3/envs/nse/lib/python3.12/site-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)


In [31]:
# nifty_candles
train_dates
# x = nifty_candles.loc[train_dates.previous_trading_day.dt.date.values][sma_key].shift(1)
# x.values

,previous_trading_day,expiry,nifty_at_0320,td_nifty_at_0918,td_nifty_at_0328,td_nifty_diff,nifty_at_buy,nifty_at_sell,nifty_diff,atm_strike,day_sma,sma_favors
trade_date,,,,,,,,,,,,
2019-09-03,2019-08-30,2019-09-05,11035.9,10960.95,11034.85,73.90,11040.95,10922.60,-118.35,11050,NaN,False
2019-09-04,2019-09-03,2019-09-05,10784.1,10790.40,10779.40,-11.00,10779.9,10799.25,19.35,10800,NaN,False
2019-09-05,2019-09-04,2019-09-05,10843.8,10860.95,10836.85,-24.10,10837.8,10883.25,45.45,10850,NaN,False
2019-09-06,2019-09-05,2019-09-12,10854.0,10883.80,10841.70,-42.10,10845.4,10890.90,45.5,10850,NaN,False
2019-09-09,2019-09-06,2019-09-12,10947.05,10936.70,10940.45,3.75,10941.15,10896.15,-45.0,10950,NaN,False
2019-09-11,2019-09-09,2019-09-12,11002.2,11028.50,11003.25,-25.25,11005.0,11032.10,27.1,11000,NaN,False
2019-09-12,2019-09-11,2019-09-12,11033.65,11058.30,11023.30,-35.00,11026.7,11065.80,39.1,11050,NaN,False
2019-09-13,2019-09-12,2019-09-19,10979.75,10986.80,10981.10,-5.70,10984.5,10975.70,-8.8,11000,NaN,False
2019-09-16,2019-09-13,2019-09-19,11080.5,10994.85,11080.70,85.85,11082.95,11022.10,-60.85,11100,10950.82,True


In [4]:
otype = OPTION_TYPE_PUT
otype = OPTION_TYPE_CALL
"""
Prev results 
C PnL at for strike: -3, Total: -469574.9999999999, Per Day: -829.6378091872789, Days: 566, Pc Mean: -5.871717218974865
C PnL at for strike: -2, Total: -516849.9999999999, Per Day: -916.400709219858, Days: 564, Pc Mean: -3.685215175381849
C PnL at for strike: -1, Total: -597075.0, Per Day: -1054.9028268551237, Days: 566, Pc Mean: -2.299273549332703
C PnL at for strike: 0, Total: -608700.0, Per Day: -1079.2553191489362, Days: 564, Pc Mean: -1.4492070709083829

First wrong run for same day sma 
C PnL at for strike: -3, Total: -20550.000000000007, Per Day: -64.42006269592478, Days: 319, Pc Mean: -3.9951029684823403
C PnL at for strike: -2, Total: -115000.00000000006, Per Day: -361.63522012578636, Days: 318, Pc Mean: -2.1037747819249057
C PnL at for strike: -1, Total: -11774.99999999988, Per Day: -36.91222570532878, Days: 319, Pc Mean: -0.8748155551964487
C PnL at for strike: 0, Total: -33575.00000000001, Per Day: -105.58176100628933, Days: 318, Pc Mean: -0.23860706089594225

After fixing everything
C PnL at for strike: -3, Total: 138375.00000000006, Per Day: 724.4764397905763, Days: 191, Pc Mean: 0.02972083357944082
C PnL at for strike: -2, Total: 179474.99999999994, Per Day: 939.6596858638741, Days: 191, Pc Mean: 0.028057874495825898
C PnL at for strike: -1, Total: 229150.00000000006, Per Day: 1199.7382198952882, Days: 191, Pc Mean: 0.026420391492207346
C PnL at for strike: 0, Total: 305950.0, Per Day: 1601.8324607329844, Days: 191, Pc Mean: 0.02653616142867675

Bad run, SMA getting same day close, invalid data 
C PnL at for strike: -3, Total: -43700.0, Per Day: -139.61661341853036, Days: 313, Pc Mean: -0.005431303947948968
C PnL at for strike: -2, Total: -130150.00000000006, Per Day: -417.1474358974361, Days: 312, Pc Mean: -0.012133076967896061
C PnL at for strike: -1, Total: -63624.999999999956, Per Day: -203.27476038338645, Days: 313, Pc Mean: -0.004344323019620291
C PnL at for strike: 0, Total: -81850.0, Per Day: -262.3397435897436, Days: 312, Pc Mean: -0.004294468871003284

After first improvement fixes, not satisfactory
C PnL at for strike: -3, Total: 10775.00000000001, Per Day: 34.53525641025644, Days: 312, Pc Mean: 0.001383712008116066
C PnL at for strike: -2, Total: -60875.0, Per Day: -195.7395498392283, Days: 311, Pc Mean: -0.005827212964890743
C PnL at for strike: -1, Total: 23749.999999999985, Per Day: 76.12179487179482, Days: 312, Pc Mean: 0.0016550609149525162
C PnL at for strike: 0, Total: 22450.000000000007, Per Day: 72.1864951768489, Days: 311, Pc Mean: 0.001197156154222509

SMA 9
C PnL at for strike: -3, Total: 75374.99999999997, Per Day: 286.59695817490484, Days: 263, Pc Mean: 0.011503594157776654
C PnL at for strike: -2, Total: 27649.99999999999, Per Day: 105.53435114503813, Days: 262, Pc Mean: 0.00316120125647746
C PnL at for strike: -1, Total: 131050.00000000004, Per Day: 498.2889733840306, Days: 263, Pc Mean: 0.0108688665650691
C PnL at for strike: 0, Total: 158099.99999999997, Per Day: 603.4351145038166, Days: 262, Pc Mean: 0.010055460859388911

SMA 5
C PnL at for strike: -3, Total: 9325.000000000027, Per Day: 39.34599156118155, Days: 237, Pc Mean: 0.001448161262268609
C PnL at for strike: -2, Total: -66625.00000000006, Per Day: -282.3093220338985, Days: 236, Pc Mean: -0.007897396081824735
C PnL at for strike: -1, Total: 14600.0, Per Day: 61.60337552742616, Days: 237, Pc Mean: 0.0012748753503724207
C PnL at for strike: 0, Total: -13599.999999999956, Per Day: -57.627118644067615, Days: 236, Pc Mean: -0.000923110338835792

Strike Results testing all_dates
C PnL at for strike: -10, Total: 31699.999999999985, Per Day: 77.69607843137251, Days: 408, Pc Mean: 2.3745318352059916
C PnL at for strike: -9, Total: 49924.999999999985, Per Day: 122.36519607843134, Days: 408, Pc Mean: 2.8072592321857814
C PnL at for strike: -8, Total: 79849.99999999997, Per Day: 195.7107843137254, Days: 408, Pc Mean: 3.3344817146376844
C PnL at for strike: -7, Total: 115500.00000000001, Per Day: 283.0882352941177, Days: 408, Pc Mean: 3.537546229297315
C PnL at for strike: -6, Total: 172575.00000000006, Per Day: 422.9779411764707, Days: 408, Pc Mean: 3.857071017488966
    C PnL at for strike: -5, Total: 247975.0, Per Day: 607.781862745098, Days: 408, Pc Mean: 4.029689455124559
C PnL at for strike: -4, Total: 328725.0, Per Day: 805.6985294117648, Days: 408, Pc Mean: 3.8831596853072496
C PnL at for strike: -3, Total: 415699.99999999994, Per Day: 1018.8725490196077, Days: 408, Pc Mean: 3.5945826172487214
C PnL at for strike: -2, Total: 506999.99999999994, Per Day: 1242.6470588235293, Days: 408, Pc: 3.2482145231066895
C PnL at for strike: -1, Total: 626950.0, Per Day: 1536.6421568627452, Days: 408, Pc: 3.0329050484723004
C PnL at for strike: 0, Total: 782949.9999999999, Per Day: 1918.9950980392155, Days: 408, Pc: 2.926010738734092
C PnL at for strike: 1, Total: 924599.9999999999, Per Day: 2266.176470588235, Days: 408, Pc: 2.7376311022843933

When min sma gap 20
C PnL at for strike: -5, Total: 265275.0, Per Day: 615.4872389791183, Days: 431, Pc: 4.0344166806077295

When min sma gap 10
C PnL at for strike: -5, Total: 317100.0, Per Day: 714.1891891891892, Days: 444, Pc: 4.690117252931323

When min sma gap 0
C PnL at for strike: -5, Total: 310650.0, Per Day: 682.7472527472528, Days: 455, Pc: 4.519416469355913

When min sma gap 5
C PnL at for strike: -5, Total: 313425.0, Per Day: 696.5, Days: 450, Pc: 4.590727002424074

When min sma gap 7
C PnL at for strike: -5, Total: 314300.0, Per Day: 703.1319910514542, Days: 447, Pc: 4.619986623646747

When min sma gap 9
C PnL at for strike: -5, Total: 314825.0, Per Day: 707.4719101123595, Days: 445, Pc: 4.651669621749409

When min sma gap 11
C PnL at for strike: -5, Total: 312400.0, Per Day: 705.1918735891647, Days: 443, Pc: 4.625869479108439
"""
for i in range(-6, 1):
    strike_key = f"cur_{otype}_atm_strike_{i}"
    train_dates[strike_key] = train_dates["atm_strike"].apply(lambda r: get_next_1000(r, i))
    buy_at = f"{otype}_at_0328_{i}"
    sell_at = f"{otype}_at_0918_{i}"
    train_dates[buy_at] = train_dates.apply(lambda trade: get_premium_df(trade, trade[strike_key], trade.previous_trading_day.date(), otype, buy_time), axis=1)
    train_dates[sell_at] = train_dates.apply(lambda trade: get_premium_df(trade, trade[strike_key], trade.name.date(), otype, sell_time), axis=1)
    pnl_key = f"{otype}_pnl_0328_{i}"
    train_dates[pnl_key] = LOT_SIZE * LOT_QTY * (train_dates.loc[train_dates[sell_at].notna()][sell_at] - train_dates.loc[train_dates[sell_at].notna()][buy_at])
    pnl_pc_key = f"{otype}_pnl_pc_{i}"
    train_dates[pnl_pc_key] = train_dates.loc[train_dates[buy_at].notna()][pnl_key] * 100 / (LOT_SIZE * LOT_QTY * train_dates.loc[train_dates[buy_at].notna()][buy_at])
    print(f"{otype} PnL at for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}, Pc: {train_dates[pnl_key].sum() * 100 / (LOT_SIZE * LOT_QTY * train_dates[buy_at].sum())}")

C PnL at for strike: -6, Total: -36549.999999999985, Per Day: -120.23026315789468, Days: 304, Pc: -1.2422254698705082
C PnL at for strike: -5, Total: 2600.0000000000564, Per Day: 8.552631578947555, Days: 304, Pc: 0.06273904178179014
C PnL at for strike: -4, Total: 2375.0000000000064, Per Day: 7.864238410596047, Days: 302, Pc: 0.04340119147691982
C PnL at for strike: -3, Total: 48450.00000000001, Per Day: 158.33333333333337, Days: 306, Pc: 0.629406740303142
C PnL at for strike: -2, Total: 65149.999999999985, Per Day: 217.89297658862873, Days: 299, Pc: 0.6316457935928915
C PnL at for strike: -1, Total: 119075.00000000003, Per Day: 392.98679867986806, Days: 303, Pc: 0.8410201717708217
C PnL at for strike: 0, Total: 161550.00000000003, Per Day: 531.4144736842106, Days: 304, Pc: 0.8739033945913195


In [5]:
"""
-5 results gap 30 or 40
Month: 2021-01-31 00:00:00, profit: -28275.0, buy mean: 15675.0, PC: -25.768967874231027
Month: 2021-02-28 00:00:00, profit: 26475.0, buy mean: 21167.307692307688, PC: 9.621150177159988
Month: 2021-03-31 00:00:00, profit: -200.0, buy mean: 20179.166666666664, PC: -0.16518686764401933
Month: 2021-04-30 00:00:00, profit: -20400.0, buy mean: 18507.5, PC: -11.02255842226125
Month: 2021-05-31 00:00:00, profit: 2950.0, buy mean: 16068.333333333336, PC: 1.2239394253708176
Month: 2021-06-30 00:00:00, profit: -2375.0, buy mean: 8449.999999999998, PC: -2.1620391442876628
Month: 2021-07-31 00:00:00, profit: -6275.0, buy mean: 4965.624999999999, PC: -15.796098174952803
Month: 2021-08-31 00:00:00, profit: 17375.0, buy mean: 5570.833333333334, PC: 17.327349788082774
Month: 2021-09-30 00:00:00, profit: 27125.0, buy mean: 11325.0, PC: 12.60601835715116
Month: 2021-10-31 00:00:00, profit: 18325.0, buy mean: 14045.000000000002, PC: 13.047347810608755
Month: 2021-11-30 00:00:00, profit: -7375.0, buy mean: 13940.000000000002, PC: -10.58106169296987
Month: 2021-12-31 00:00:00, profit: 15275.0, buy mean: 20233.33333333333, PC: 8.388248215266337
Month: 2022-01-31 00:00:00, profit: 23525.0, buy mean: 18945.83333333333, PC: 10.347481856168903
Month: 2022-02-28 00:00:00, profit: -19075.0, buy mean: 22687.5, PC: -14.012855831037646
Month: 2022-03-31 00:00:00, profit: 10900.0, buy mean: 41614.28571428571, PC: 1.8709234466186062
Month: 2022-04-30 00:00:00, profit: -19325.0, buy mean: 25528.571428571428, PC: -10.814213766088416
Month: 2022-05-31 00:00:00, profit: 53475.0, buy mean: 29015.625000000004, PC: 23.037156704361863
Month: 2022-06-30 00:00:00, profit: 17100.0, buy mean: 22411.111111111113, PC: 8.477937530986607
Month: 2022-07-31 00:00:00, profit: 24900.0, buy mean: 18835.29411764706, PC: 7.776389756402253
Month: 2022-08-31 00:00:00, profit: 7300.0, buy mean: 17490.384615384617, PC: 3.2105552501374395
Month: 2022-09-30 00:00:00, profit: 2875.0, buy mean: 17788.888888888887, PC: 1.7957526545908822
Month: 2022-10-31 00:00:00, profit: 1550.0, buy mean: 18593.750000000004, PC: 0.6946778711484592
Month: 2022-11-30 00:00:00, profit: -11025.0, buy mean: 13051.5625, PC: -5.279540284927573
Month: 2022-12-31 00:00:00, profit: -9675.0, buy mean: 16399.999999999996, PC: -11.798780487804878
Month: 2023-01-31 00:00:00, profit: -2800.0, buy mean: 6917.857142857143, PC: -5.782137325761485
Month: 2023-02-28 00:00:00, profit: -23750.0, buy mean: 11140.625, PC: -26.647966339410935
Month: 2023-03-31 00:00:00, profit: 9100.0, buy mean: 9045.833333333336, PC: 16.766467065868255
Month: 2023-04-30 00:00:00, profit: 9650.0, buy mean: 5417.3076923076915, PC: 13.702520411785587
Month: 2023-05-31 00:00:00, profit: 14750.0, buy mean: 6779.6875, PC: 13.597603134362753
Month: 2023-06-30 00:00:00, profit: 18375.0, buy mean: 5766.071428571429, PC: 22.7624651594921
Month: 2023-07-31 00:00:00, profit: 15825.0, buy mean: 9010.0, PC: 11.709211986681463
Month: 2023-08-31 00:00:00, profit: -150.0, buy mean: 1000.0, PC: -15.000000000000004
Month: 2023-09-30 00:00:00, profit: 9325.0, buy mean: 7293.749999999999, PC: 10.654098828906024
Month: 2023-10-31 00:00:00, profit: -13450.0, buy mean: 9282.142857142859, PC: -20.70026933435937
Month: 2023-11-30 00:00:00, profit: -275.0, buy mean: 8307.352941176468, PC: -0.19472473004071456
Month: 2023-12-31 00:00:00, profit: 68350.0, buy mean: 20553.947368421053, PC: 17.50208053261635
Month: 2024-01-31 00:00:00, profit: -5175.0, buy mean: 25369.444444444445, PC: -2.2665060768641205
Month: 2024-02-29 00:00:00, profit: 23050.0, buy mean: 27983.33333333333, PC: 27.456819535437756
Year: 2021-12-31 00:00:00, profit: 42625.0, buy mean: 13453.75939849624, PC: 2.3821499427166293
Year: 2022-12-31 00:00:00, profit: 82525.0, buy mean: 21720.5078125, PC: 2.968284940967008
Year: 2023-12-31 00:00:00, profit: 104950.0, buy mean: 9421.111111111111, PC: 8.251759248339035
Year: 2024-12-31 00:00:00, profit: 17875.0, buy mean: 26022.916666666664, PC: 5.724121367384513

Gap 11
Month: 2021-01-31 00:00:00, profit: -28275.0, buy mean: 15675.0, PC: -25.768967874231027
Month: 2021-02-28 00:00:00, profit: 24675.0, buy mean: 19898.214285714283, PC: 8.857578748990393
Month: 2021-03-31 00:00:00, profit: 10750.0, buy mean: 23465.625, PC: 5.726461579438007
Month: 2021-04-30 00:00:00, profit: -20400.0, buy mean: 18507.5, PC: -11.02255842226125
Month: 2021-05-31 00:00:00, profit: 10375.0, buy mean: 17207.812500000004, PC: 3.7682738581676243
Month: 2021-06-30 00:00:00, profit: -850.0, buy mean: 8786.666666666664, PC: -0.6449165402124429
Month: 2021-07-31 00:00:00, profit: -6525.0, buy mean: 4650.000000000001, PC: -15.591397849462366
Month: 2021-08-31 00:00:00, profit: 17950.0, buy mean: 5618.421052631579, PC: 16.814988290398126
Month: 2021-09-30 00:00:00, profit: 23900.0, buy mean: 11045.0, PC: 10.819375282933459
Month: 2021-10-31 00:00:00, profit: 14625.0, buy mean: 16225.000000000002, PC: 7.511556240369798
Month: 2021-11-30 00:00:00, profit: -7375.0, buy mean: 13940.000000000002, PC: -10.58106169296987
Month: 2021-12-31 00:00:00, profit: 36700.0, buy mean: 19280.0, PC: 19.03526970954357
Month: 2022-01-31 00:00:00, profit: 14100.0, buy mean: 18653.846153846152, PC: 5.814432989690722
Month: 2022-02-28 00:00:00, profit: -19075.0, buy mean: 22687.5, PC: -14.012855831037646
Month: 2022-03-31 00:00:00, profit: 10900.0, buy mean: 41614.28571428571, PC: 1.8709234466186062
Month: 2022-04-30 00:00:00, profit: -17150.0, buy mean: 27640.625, PC: -7.755794234030524
Month: 2022-05-31 00:00:00, profit: 53475.0, buy mean: 29015.625000000004, PC: 23.037156704361863
Month: 2022-06-30 00:00:00, profit: 45950.0, buy mean: 25037.5, PC: 18.352471293060404
Month: 2022-07-31 00:00:00, profit: 44175.0, buy mean: 18613.15789473684, PC: 12.491163579810552
Month: 2022-08-31 00:00:00, profit: -6325.0, buy mean: 18071.428571428572, PC: -2.4999999999999987
Month: 2022-09-30 00:00:00, profit: -12800.0, buy mean: 20100.000000000004, PC: -6.368159203980097
Month: 2022-10-31 00:00:00, profit: -6250.0, buy mean: 17911.53846153847, PC: -2.6841314150740825
Month: 2022-11-30 00:00:00, profit: -21250.0, buy mean: 13304.411764705883, PC: -9.395379683873111
Month: 2022-12-31 00:00:00, profit: -11750.0, buy mean: 14125.0, PC: -13.864306784660767
Month: 2023-01-31 00:00:00, profit: 1550.0, buy mean: 7643.75, PC: 2.5347506132461177
Month: 2023-02-28 00:00:00, profit: -21375.0, buy mean: 11688.888888888889, PC: -20.318441064638783
Month: 2023-03-31 00:00:00, profit: 9100.0, buy mean: 9045.833333333336, PC: 16.766467065868255
Month: 2023-04-30 00:00:00, profit: 9650.0, buy mean: 5417.3076923076915, PC: 13.702520411785587
Month: 2023-05-31 00:00:00, profit: 17150.0, buy mean: 6386.111111111111, PC: 14.919530230535015
Month: 2023-06-30 00:00:00, profit: 24550.0, buy mean: 7973.611111111111, PC: 17.105033966207976
Month: 2023-07-31 00:00:00, profit: 19575.0, buy mean: 8313.235294117647, PC: 13.851052538475143
Month: 2023-08-31 00:00:00, profit: 450.0, buy mean: 1050.0, PC: 21.42857142857142
Month: 2023-09-30 00:00:00, profit: 9325.0, buy mean: 7293.749999999999, PC: 10.654098828906024
Month: 2023-10-31 00:00:00, profit: -13450.0, buy mean: 9282.142857142859, PC: -20.70026933435937
Month: 2023-11-30 00:00:00, profit: 9825.0, buy mean: 9072.222222222223, PC: 6.016533986527861
Month: 2023-12-31 00:00:00, profit: 68350.0, buy mean: 20553.947368421053, PC: 17.50208053261635
Month: 2024-01-31 00:00:00, profit: 5100.0, buy mean: 25889.999999999996, PC: 1.9698725376593276
Month: 2024-02-29 00:00:00, profit: 23050.0, buy mean: 27983.33333333333, PC: 27.456819535437756
Year: 2021-12-31 00:00:00, profit: 75550.0, buy mean: 13758.103448275862, PC: 3.787109791094903
Year: 2022-12-31 00:00:00, profit: 74000.0, buy mean: 21857.065217391304, PC: 2.453357203835857
Year: 2023-12-31 00:00:00, profit: 134700.0, buy mean: 9518.87755102041, PC: 9.626413678512087
Year: 2024-12-31 00:00:00, profit: 28150.0, buy mean: 26373.07692307692, PC: 8.21058772057751
"""
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 2000)

num = 0
num = -5
pnl_key = f"{otype}_pnl_0328_{num}"
buy_at_key = f"{otype}_at_0328_{num}"
train_dates["dd"] = train_dates[pnl_key]

for idx, row in train_dates.iterrows():
    pr = train_dates.shift(1).loc[idx]
    if pd.isna(pr.atm_strike) or pd.isnull(pr.atm_strike):
        continue
    train_dates.loc[idx, "dd"] = pr.dd + row[pnl_key]

def format_float(value):
    return f'{value:.2f}'

train_dates["trade_date"] = train_dates.index.values
gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="ME"))
for month, mdf in gdf:
    print(f"Month: {month}, profit: {round(mdf[pnl_key].sum(), 2)}, buy mean: {mdf[buy_at_key].mean() * LOT_SIZE * LOT_QTY}, PC: {mdf[pnl_key].sum() * 100 / (LOT_SIZE * LOT_QTY * mdf[buy_at_key].sum())}")

gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="YE"))
for year, mdf in gdf:
    print(f"Year: {year}, profit: {round(mdf[pnl_key].sum(), 2)}, buy mean: {mdf[buy_at_key].mean() * LOT_SIZE * LOT_QTY}, PC: {mdf[pnl_key].sum() * 100 / (LOT_SIZE * LOT_QTY * mdf[buy_at_key].sum())}")


pd.options.display.float_format = format_float

# Set the display.float_format option to use the formatting function
train_dates[train_dates[f"{otype}_pnl_0328_{num}"].notna()][["previous_trading_day", "expiry", "nifty_at_0320", "atm_strike", f"cur_{otype}_atm_strike_{num}", f"{otype}_at_0328_{num}", f"{otype}_at_0918_{num}", pnl_key, "nifty_diff", "td_nifty_at_0918", "td_nifty_at_0328", f"{otype}_pnl_pc_{num}", "dd", "day_sma", "sma_favors"]]

Month: 2018-09-30 00:00:00, profit: -11450.0, buy mean: 18675.0, PC: -61.311914323962526
Month: 2018-10-31 00:00:00, profit: 2200.0, buy mean: 18241.666666666668, PC: 4.0201005025125625
Month: 2018-11-30 00:00:00, profit: 23750.0, buy mean: 29582.14285714286, PC: 5.734637208740795
Month: 2018-12-31 00:00:00, profit: -14275.0, buy mean: 31645.83333333333, PC: -3.759052007899932
Month: 2019-01-31 00:00:00, profit: -8375.0, buy mean: 30040.0, PC: -2.787949400798934
Month: 2019-02-28 00:00:00, profit: -1125.0, buy mean: 2780.0, PC: -8.093525179856115
Month: 2019-03-31 00:00:00, profit: 6700.0, buy mean: 4644.117647058823, PC: 8.486383787207094
Month: 2019-04-30 00:00:00, profit: -2500.0, buy mean: 3764.583333333333, PC: -5.534034311012725
Month: 2019-05-31 00:00:00, profit: 29300.0, buy mean: 27980.769230769227, PC: 8.054982817869416
Month: 2019-06-30 00:00:00, profit: -5675.0, buy mean: 3353.1249999999995, PC: -21.155638397017707
Month: 2019-07-31 00:00:00, profit: 2875.0, buy mean: 3918.

,previous_trading_day,expiry,nifty_at_0320,atm_strike,cur_C_atm_strike_-5,C_at_0328_-5,C_at_0918_-5,C_pnl_0328_-5,nifty_diff,td_nifty_at_0918,td_nifty_at_0328,C_pnl_pc_-5,dd,day_sma,sma_favors
trade_date,,,,,,,,,,,,,,,
2018-09-17,2018-09-14,2018-09-27,11518.25,11500,11700,37.35,14.45,-11450.00,-90.05,11464.95,11512.80,-61.31,-11450.00,11479.59,True
2018-10-16,2018-10-15,2018-10-25,10516.80,10500,10700,53.15,54.50,675.00,12.50,10550.15,10519.20,2.54,-10775.00,10455.87,True
2018-10-17,2018-10-16,2018-10-25,10591.20,10600,10800,29.50,49.15,9825.00,125.25,10688.70,10576.60,66.61,-950.00,10425.48,True
2018-10-19,2018-10-17,2018-10-25,10454.10,10450,10650,26.80,10.20,-8300.00,-137.30,10339.70,10451.95,-61.94,-9250.00,10409.23,True
2018-11-01,2018-10-31,2018-11-29,10388.10,10400,10600,105.00,116.70,5850.00,41.10,10441.70,10372.60,11.14,-3400.00,10212.34,True
2018-11-02,2018-11-01,2018-11-29,10376.40,10400,10600,106.55,130.55,12000.00,78.90,10462.30,10381.15,22.52,8600.00,10220.89,True
2018-11-05,2018-11-02,2018-11-29,10548.40,10550,10750,103.50,83.55,-9975.00,-33.20,10558.75,10553.85,-19.28,-1375.00,10255.08,True
2018-11-06,2018-11-05,2018-11-29,10531.95,10550,10750,89.50,96.10,3300.00,42.20,10552.00,10524.90,7.37,1925.00,10296.99,True
2018-11-12,2018-11-09,2018-11-29,10587.65,10600,10800,61.75,64.25,1250.00,39.30,10607.80,10580.30,4.05,3175.00,10445.21,True


In [8]:
print(train_dates.loc[(train_dates.index.year==2018) & (train_dates.nifty_diff > 0)].shape)
print(train_dates.loc[(train_dates.index.year==2018) & (train_dates.nifty_diff < 0)].shape)

(50, 32)
(33, 32)


In [5]:
train_dates.to_csv(f"type-{otype}-strike-i{num}-expiry-{expiry}-buy-at-{buy_time}-sell-time-{sell_time}.csv")

In [3]:
"""
Results all_dates i=-1 expiry 1st

Month: 2021-01-31 00:00:00, profit: 139605.0
Month: 2021-02-28 00:00:00, profit: 71115.0
Month: 2021-03-31 00:00:00, profit: 16797.5
Month: 2021-04-30 00:00:00, profit: 126210.0
Month: 2021-05-31 00:00:00, profit: 34687.5
Month: 2021-06-30 00:00:00, profit: 47577.5
Month: 2021-07-31 00:00:00, profit: 8377.5
Month: 2021-08-31 00:00:00, profit: 432315.0
Month: 2021-09-30 00:00:00, profit: 216330.0
Month: 2021-10-31 00:00:00, profit: 334595.0
Month: 2021-11-30 00:00:00, profit: -63745.0
Month: 2021-12-31 00:00:00, profit: 391830.0
Month: 2022-01-31 00:00:00, profit: -192085.0
Month: 2022-02-28 00:00:00, profit: -70482.5
Month: 2022-03-31 00:00:00, profit: 92437.5
Month: 2022-04-30 00:00:00, profit: -108052.5
Month: 2022-05-31 00:00:00, profit: -42137.5
Month: 2022-06-30 00:00:00, profit: -240587.5
Month: 2022-07-31 00:00:00, profit: 418250.0
Month: 2022-08-31 00:00:00, profit: 83580.0
Month: 2022-09-30 00:00:00, profit: -42612.5
Month: 2022-10-31 00:00:00, profit: 324885.0
Month: 2022-11-30 00:00:00, profit: -49162.5
Month: 2022-12-31 00:00:00, profit: -125740.0
Month: 2023-01-31 00:00:00, profit: -59820.0
Month: 2023-02-28 00:00:00, profit: -134817.5
Month: 2023-03-31 00:00:00, profit: 156280.0
Month: 2023-04-30 00:00:00, profit: -48010.0
Month: 2023-05-31 00:00:00, profit: 188342.5
Month: 2023-06-30 00:00:00, profit: 104335.0
Month: 2023-07-31 00:00:00, profit: 315392.5
Month: 2023-08-31 00:00:00, profit: -118622.5
Month: 2023-09-30 00:00:00, profit: -151150.0
Month: 2023-10-31 00:00:00, profit: -229855.0
Month: 2023-11-30 00:00:00, profit: 330382.5
Month: 2023-12-31 00:00:00, profit: 599397.5
Year: 2021-12-31 00:00:00, profit: 1755695.0
Year: 2022-12-31 00:00:00, profit: 48292.5
Year: 2023-12-31 00:00:00, profit: 951855.0
"""
pnl_key = f"call_pnl_0918_{num}"
train_dates["trade_date"] = train_dates.index.values
gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="ME"))
for month, mdf in gdf:
    print(f"Month: {month}, profit: {round(mdf[pnl_key].sum(), 2)}, max dd: {mdf["dd"].min()}, max idd: {mdf["dd"].idxmin()}")

gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="YE"))
for year, mdf in gdf:
    print(f"Year: {year}, profit: {round(mdf[pnl_key].sum(), 2)}, max dd: {mdf["dd"].min()}, max idd: {mdf["dd"].idxmin()}")


NameError: name 'num' is not defined

In [23]:
"""
Training dataset 1st expiry, i=0
-261950.0
2021-03-31 00:00:00

Test dataset 1st expiry i=0
18250.0
2021-01-04 00:00:00

All dataset 1st expiry i=-1:
-5765.0
2021-01-05 00:00:00
"""

print(train_dates["dd"].min())
print(train_dates["dd"].idxmin())

-215725.0
2021-03-24 00:00:00


In [11]:
train_dates = train_dates.to_csv("gapup_results.csv")
# train_dates = pd.read_csv("gapup_results.csv")

## Buy puts during the market hours

In [2]:
def get_premium_df(trade, strike_price, td, option_type, tm):
    pr = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=trade.expiry, cur_dt=td, strike_price=strike_price, option_type=option_type)
    if type(pr) == type(pd.NA) or pr.shape[0] == 0:
        return pd.NA
    x = pr.loc[(pr.index.date == td) & (pr.index.time >= tm)]
    # # print(x.iloc[0])
    if x.shape[0] == 0:
        # print(tm)
        # print(td)
        # print(pr.loc[(pr.index.date == td)])
        # print(trade)
        return pd.NA
    return x.iloc[0].close

def get_next_1000(strike, i):
    divider = 100
    reminder = strike % divider
    if reminder != 0:
        return divider * (strike // divider) - i * divider
    return divider * (strike // divider) - (i+1) * divider

last_strike = None
"""
put PnL at 3:28 for strike: -2, Total: -1016200.0, Per Day: -2768.9373297002726, Days: 367, Pc Mean: -2.6088599695693726
put PnL at 3:28 for strike: -1, Total: -818350.0000000001, Per Day: -2229.8365122615805, Days: 367, Pc Mean: -2.877046907395316
put PnL at 3:28 for strike: 0, Total: -722299.9999999999, Per Day: -1962.7717391304345, Days: 368, Pc Mean: -3.1501864396409993
put PnL at 3:28 for strike: 1, Total: -597700.0, Per Day: -1624.1847826086957, Days: 368, Pc Mean: -3.382883849858163
put PnL at 3:28 for strike: 2, Total: -503849.99999999994, Per Day: -1369.157608695652, Days: 368, Pc Mean: -3.6115974299990623

Training dataset, put long
put PnL at 3:28 for strike: -9, Total: -1852500.0, Per Day: -16840.909090909092, Days: 110, Pc Mean: -4.439315720992933
put PnL at 3:28 for strike: -8, Total: -1424925.0000000002, Per Day: -9693.367346938778, Days: 147, Pc Mean: -2.863027404233216
put PnL at 3:28 for strike: -7, Total: -2490975.0, Per Day: -11479.147465437789, Days: 217, Pc Mean: -3.8930789152502503
put PnL at 3:28 for strike: -6, Total: -2475800.0, Per Day: -9413.688212927756, Days: 263, Pc Mean: -3.792994903729857
put PnL at 3:28 for strike: -5, Total: -1488850.0000000002, Per Day: -4756.709265175719, Days: 313, Pc Mean: -2.347833438133472

Training dataset call short
C PnL at 3:28 for strike: -9, Total: 166100.0, Per Day: 451.35869565217394, Days: 368, Pc Mean: 26.41369376188105
C PnL at 3:28 for strike: -8, Total: 187325.0, Per Day: 509.0353260869565, Days: 368, Pc Mean: 26.878474821106145
C PnL at 3:28 for strike: -7, Total: 199400.0, Per Day: 541.8478260869565, Days: 368, Pc Mean: 26.888533120479664
C PnL at 3:28 for strike: -6, Total: 190100.0, Per Day: 516.5760869565217, Days: 368, Pc Mean: 26.149806406662552
C PnL at 3:28 for strike: -5, Total: 143875.0, Per Day: 390.9646739130435, Days: 368, Pc Mean: 24.57488719633468

C PnL at 3:28 for strike: 4, Total: 325724.99999999977, Per Day: 1119.3298969072157, Days: 291, Pc Mean: 0.015990427098674515
C PnL at 3:28 for strike: 5, Total: 1517700.0000000002, Per Day: 6144.534412955467, Days: 247, Pc Mean: 0.08777906304222093
C PnL at 3:28 for strike: 6, Total: 885475.0000000002, Per Day: 4587.953367875649, Days: 193, Pc Mean: 0.06554219096965225
C PnL at 3:28 for strike: 7, Total: 685175.0, Per Day: 4859.397163120568, Days: 141, Pc Mean: 0.069419959473151
C PnL at 3:28 for strike: 8, Total: 272924.99999999994, Per Day: 2183.3999999999996, Days: 125, Pc Mean: 0.03119142857142857
"""
# for i in range(-15, 30):
# for i in range(5, 10):
# for i in range(10, 15):
# for i in range(-10, -5):
# for i in range(-5, 0):
# for i in range(0, 5):
otype = OPTION_TYPE_PUT
otype = OPTION_TYPE_CALL
for i in range(0, 4):
    strike_key = f"cur_{otype}_atm_strike_{i}"
    train_dates[strike_key] = train_dates["atm_strike"].apply(lambda r: get_next_1000(r, i))
    buy_at = f"{otype}_at_0918_{i}"
    sell_at = f"{otype}_at_0328_{i}"
    train_dates[sell_at] = train_dates.apply(lambda trade: get_premium_df(trade, trade[strike_key], trade.name.date(), otype, dt.time(hour=9, minute=16)), axis=1)
    train_dates[buy_at] = train_dates.loc[train_dates[sell_at].notna()].apply(lambda trade: get_premium_df(trade, trade[strike_key], trade.name.date(), otype, dt.time(hour=15, minute=28)), axis=1)
    pnl_key = f"{otype}_pnl_0328_{i}"
    train_dates[pnl_key] = LOT_SIZE * LOT_QTY * (train_dates.loc[train_dates[sell_at].notna()][sell_at] - train_dates.loc[train_dates[sell_at].notna()][buy_at])
    pnl_pc_key = f"{otype}_pnl_pc_{i}"
    # train_dates[pnl_pc_key] = train_dates.loc[train_dates[buy_at].notna()][pnl_key] * 100 / (LOT_SIZE * LOT_QTY * train_dates.loc[train_dates[buy_at].notna()][buy_at])
    train_dates[pnl_pc_key] = train_dates.loc[train_dates[buy_at].notna()][pnl_key] * 100 / (LOT_SIZE * 140000)
    print(f"{otype} PnL at 3:28 for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}, Pc Mean: {train_dates[pnl_pc_key].mean()}")


C PnL at 3:28 for strike: 0, Total: -183350.0000000001, Per Day: -499.5912806539512, Days: 367, Pc Mean: -0.0071370182950564465
C PnL at 3:28 for strike: 1, Total: -177474.99999999997, Per Day: -486.2328767123287, Days: 365, Pc Mean: -0.006946183953033275
C PnL at 3:28 for strike: 2, Total: -94275.0, Per Day: -266.3135593220339, Days: 354, Pc Mean: -0.0038044794188862164
C PnL at 3:28 for strike: 3, Total: 459400.0, Per Day: 1404.8929663608562, Days: 327, Pc Mean: 0.020069899519440756


In [8]:
num = 3
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 2000)
pnl_key = f"{otype}_pnl_0328_{num}"
train_dates["dd"] = train_dates[pnl_key]

for idx, row in train_dates[train_dates[f"{otype}_pnl_0328_{num}"].notna()].iterrows():
    pr = train_dates[train_dates[f"{otype}_pnl_0328_{num}"].notna()].shift(1).loc[idx]
    if pd.isna(pr.atm_strike) or pd.isnull(pr.atm_strike):
        continue
    train_dates.loc[idx, "dd"] = pr.dd + row[pnl_key]

def format_float(value):
    return f'{value:.2f}'

# Set the display.float_format option to use the formatting function
pd.options.display.float_format = format_float
train_dates.style.set_sticky(axis="columns")

train_dates[train_dates[f"{otype}_pnl_0328_{num}"].notna()][["previous_trading_day", "expiry", "nifty_at_0320", "atm_strike", f"cur_{otype}_atm_strike_{num}", f"{otype}_at_0918_{num}", f"{otype}_at_0328_{num}", pnl_key, "td_nifty_diff", "td_nifty_at_0918", "td_nifty_at_0328", f"{otype}_pnl_pc_{num}", "dd"]]

,previous_trading_day,expiry,nifty_at_0320,atm_strike,cur_C_atm_strike_3,C_at_0918_3,C_at_0328_3,C_pnl_0328_3,td_nifty_diff,td_nifty_at_0918,td_nifty_at_0328,C_pnl_pc_3,dd
trade_date,,,,,,,,,,,,,
2021-01-05,2021-01-04,2021-01-14,14129.55,14150,13800,439.25,356.00,-41625.00,123.50,14075.15,14198.65,-0.59,-41625.00
2021-01-12,2021-01-11,2021-01-21,14483.75,14500,14100,537.30,426.25,-55525.00,96.75,14473.80,14570.55,-0.79,-97150.00
2021-01-14,2021-01-13,2021-01-21,14566.60,14550,14200,448.35,415.30,-16525.00,44.55,14550.05,14594.60,-0.24,-113675.00
2021-01-15,2021-01-14,2021-01-28,14593.15,14600,14200,387.05,499.95,56450.00,-160.85,14594.35,14433.50,0.81,-57225.00
2021-01-18,2021-01-15,2021-01-28,14441.85,14450,14100,307.95,416.45,54250.00,-223.95,14453.30,14229.35,0.78,-2975.00
2021-01-22,2021-01-21,2021-02-04,14621.60,14600,14200,364.70,502.85,69075.00,-223.70,14583.40,14359.70,0.99,66100.00
2021-01-28,2021-01-27,2021-02-04,13971.30,13950,13600,355.00,389.55,17275.00,10.65,13810.40,13821.05,0.25,83375.00
2021-02-03,2021-02-02,2021-02-11,14651.85,14650,14300,548.00,497.05,-25475.00,31.65,14754.90,14786.55,-0.36,57900.00
2021-02-04,2021-02-03,2021-02-11,14787.50,14800,14400,501.00,419.20,-40900.00,97.40,14789.05,14886.45,-0.58,17000.00


In [9]:
# 1154.999999999991 for 918 to 328
train_dates.td_nifty_diff.sum()

-3345.149999999996

In [10]:
num = 3
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 2000)
pnl_key = f"{otype}_pnl_0328_{num}"
pnl_key = "td_nifty_diff"
train_dates["dd"] = train_dates[pnl_key]

for idx, row in train_dates[train_dates[pnl_key].notna()].iterrows():
    pr = train_dates[train_dates[pnl_key].notna()].shift(1).loc[idx]
    if pd.isna(pr.atm_strike) or pd.isnull(pr.atm_strike):
        continue
    train_dates.loc[idx, "dd"] = pr.dd + row[pnl_key]

def format_float(value):
    return f'{value:.2f}'

# Set the display.float_format option to use the formatting function
pd.options.display.float_format = format_float
train_dates.style.set_sticky(axis="columns")

train_dates[train_dates[f"{otype}_pnl_0328_{num}"].notna()][["previous_trading_day", "expiry", "nifty_at_0320", "atm_strike", f"cur_{otype}_atm_strike_{num}", f"{otype}_at_0918_{num}", f"{otype}_at_0328_{num}", pnl_key, "td_nifty_diff", "td_nifty_at_0918", "td_nifty_at_0328", f"{otype}_pnl_pc_{num}", "dd"]]

,previous_trading_day,expiry,nifty_at_0320,atm_strike,cur_C_atm_strike_3,C_at_0918_3,C_at_0328_3,td_nifty_diff,td_nifty_diff,td_nifty_at_0918,td_nifty_at_0328,C_pnl_pc_3,dd
trade_date,,,,,,,,,,,,,
2021-01-05,2021-01-04,2021-01-14,14129.55,14150,13800,439.25,356.00,123.50,123.50,14075.15,14198.65,-0.59,142.40
2021-01-12,2021-01-11,2021-01-21,14483.75,14500,14100,537.30,426.25,96.75,96.75,14473.80,14570.55,-0.79,251.65
2021-01-14,2021-01-13,2021-01-21,14566.60,14550,14200,448.35,415.30,44.55,44.55,14550.05,14594.60,-0.24,296.20
2021-01-15,2021-01-14,2021-01-28,14593.15,14600,14200,387.05,499.95,-160.85,-160.85,14594.35,14433.50,0.81,135.35
2021-01-18,2021-01-15,2021-01-28,14441.85,14450,14100,307.95,416.45,-223.95,-223.95,14453.30,14229.35,0.78,-88.60
2021-01-22,2021-01-21,2021-02-04,14621.60,14600,14200,364.70,502.85,-223.70,-223.70,14583.40,14359.70,0.99,-312.30
2021-01-28,2021-01-27,2021-02-04,13971.30,13950,13600,355.00,389.55,10.65,10.65,13810.40,13821.05,0.25,-301.65
2021-02-03,2021-02-02,2021-02-11,14651.85,14650,14300,548.00,497.05,31.65,31.65,14754.90,14786.55,-0.36,-270.00
2021-02-04,2021-02-03,2021-02-11,14787.50,14800,14400,501.00,419.20,97.40,97.40,14789.05,14886.45,-0.58,-172.60


In [12]:
train_dates["trade_date"] = train_dates.index.values
gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="ME"))
for month, mdf in gdf:
    print(f"Month: {month}, profit: {round(mdf[pnl_key].sum(), 2)}, max dd: {mdf["dd"].max()}, max idd: {mdf["dd"].idxmin()}")

gdf = train_dates.groupby(pd.Grouper(key="trade_date", freq="YE"))
for year, mdf in gdf:
    print(f"Year: {year}, profit: {round(mdf[pnl_key].sum(), 2)}, max dd: {mdf["dd"].max()}, max idd: {mdf["dd"].idxmin()}")


Month: 2021-01-31 00:00:00, profit: -301.65, max dd: 296.2000000000007, max idd: 2021-01-22 00:00:00
Month: 2021-02-28 00:00:00, profit: -305.9, max dd: -47.19999999999709, max idd: 2021-02-26 00:00:00
Month: 2021-03-31 00:00:00, profit: -62.0, max dd: -278.59999999999854, max idd: 2021-03-24 00:00:00
Month: 2021-04-30 00:00:00, profit: 225.65, max dd: -213.79999999999563, max idd: 2021-04-20 00:00:00
Month: 2021-05-31 00:00:00, profit: 209.45, max dd: -234.4499999999971, max idd: 2021-05-04 00:00:00
Month: 2021-06-30 00:00:00, profit: 137.6, max dd: 79.15000000000146, max idd: 2021-06-01 00:00:00
Month: 2021-07-31 00:00:00, profit: -96.6, max dd: 20.60000000000764, max idd: 2021-07-28 00:00:00
Month: 2021-08-31 00:00:00, profit: 316.05, max dd: 122.600000000004, max idd: 2021-08-18 00:00:00
Month: 2021-09-30 00:00:00, profit: -152.1, max dd: 345.95000000000255, max idd: 2021-09-30 00:00:00
Month: 2021-10-31 00:00:00, profit: -1128.65, max dd: -33.74999999999818, max idd: 2021-10-29 00